# ASCON

This ASCON implementation supports the following ciphers:

- ASCON-128
- ASCON-128a
- ASCON-Hash
- ASCON-Xof

Input data can either be any utf-8 encoded string or any hex-string.

The following code defines the following classes and Exceptions:
- EngineError(Exception) - indicates error in Ascon-Engine
- EngineTimeOut(Exception) - indicates time out
- AuthenticatoinError(Exception) - indicates invalid tag on decryption
- AsconCipher - Ciphertext representation for Ascon-Engine
- AsconPlain - Plaintext representation for Ascon-Engine
- AsconKey - Keydata representation for Ascon-Engine



The following code block defines relevant import statements

In [225]:
import secrets
from dataclasses import dataclass

from pynq import Overlay
from pynq import MMIO


The following code block define exceptions raised by Ascon-Engine

In [217]:
# Exceptions for ASCON Engine

class EngineError(Exception):
    """ Raised when Engine indicated an error. """
    pass

class EngineTimeOut(Exception):
    """ Raised when Engine does not responde. """
    pass

class AuthenticationError(Exception):
    """ Raised when authentication failed. """
    pass

The following code block define dataclasses to represent cipher, plain and key material.

In [218]:
@dataclass
class AsconCipher():
    """
    Models a cipher text from an ascon encryption 
    
    Attributes:
        - cipher: Ciphertext as hex string
        - tag: Tag as hex string
        - ad_data: Associated data as hex string
        - cipher_t: Hex string with last part of cipher
        - is_hex: Is ad_data a hex string?
    """
    cipher: str
    tag: str
    ad_data: str=None
    cipher_t: str = ""
    is_hex: bool=False

@dataclass
class AsconPlain():
    """
    Models a plaintext element.
    
    Attributes:
        - plain: Plaintext as strinng (utf-8 or hex data)
        - ad_data: Associated data as string (utf-8 or hex data)
        - is_hex: True when plain and associated data are hex string
        
    Note:
    Plain and ad_data must either be a normal string (utf-8) e.g. 'My plaintext' or 
    a string of hex characters without leading 0x e.g. '01020304AFEF'
    """
    plain: str
    ad_data: str = None
    is_hex: bool = False


@dataclass
class AsconKey:
    """
    Models a key and nonce element for ASCON 
    
    Attributes:
        -key: hex string of key (32 bytes)
        -nonce hex string of nonce (32 bytes)

    Notes:
        Key and Nonce are assigned randomly if not set.
    """
    key: int = secrets.token_hex(128//8)
    nonce: int = secrets.token_hex(128//8)

The following code block defines the Ascon Engine

In [219]:
class Ascon():
    
    """
    Class implementing a ASCON HW-Accelerator
    """
    BITSTREAM = "./bitstreams/AsconEngine.bit"
    BLOCK_SIZE_64 = 64//4 # length of 64 bits as hex-string representation
    BLOCK_SIZE_128 = 128//4
    
    KEY = 0
    NONCE = 1
    AD = 2
    PLAIN = 3
    CIPHER = 4
    TAG = 5
    MSG = 6
    HASH = 7
    LEN = 8
    CONF = 9
    ABORT = 10
    OK = 11
    SKIP_AD =12

    def __init__(self):
        """
        Constructor for class ASCON
        """
        
        self._overlay = None
        self._toggle_2pl = 0
        self._toggle_2ps = 0
        self._buffer2pl = None
        self._buffer2ps = None
        
        self.hard_reset()

    # STATIC METHODS
    @staticmethod
    def hex_to_utf8(hex_str):
        """
        Encodes a hex-string as utf-8 string
        
        Parameters:
            - hex_str: String with hex characters
        
        Returns:
            UTF-8 representation of hex data

        """
        byte_data = bytes.fromhex(hex_str)
        utf8_string = byte_data.decode('utf-8')

        return utf8_string

    @staticmethod
    def hex_and_pad(msg: str, block_size: int) -> str:
        """
        Encodes string as hex and adds padding.
        
        Parameters:
            - msg: String message of arbitrary length
            - block_size: Valid block size for padding
        
        Returns:
            Paddes hex string for ASCON Engine.
        """
        hex_str = msg.encode().hex()
        hex_str += '8'
        length = len(hex_str)
        pad_len = (block_size - (length % block_size))
        hex_str += '0' * pad_len

        return hex_str, (pad_len+1)
    
    @staticmethod
    def add_pad(msg: str, block_size: int) -> str:
        """
        Adds padding to hex string.
        
        Parameters:
            - msg: String message of arbitrary length
            - block_size: Valid block size for padding
        
        Returns:
            Paddes hex string for ASCON Engine.
        """
        hex_str = msg
        hex_str += '8'
        length = len(hex_str)
        pad_len = (block_size - (length % block_size))
        hex_str += '0' * pad_len

        return hex_str, (pad_len+1)

    
    # PRIVATE METHODS FOR INTERNAL USE ONLY
    def _set_toggle(self) -> None:
        """
        Toggles last received toggled bit.
        """
        self._toggle_2ps = 0#self._buffer2ps.read() >>2 & 0x1
        self._toggle_2pl = 0
        
    def _wait_toggle(self, auth: bool=False) -> int:
        """
        Waits for toggle to change
        
        ToDo: Implement a timeout.
        
        Parameters:
            - auth: Check authentication flag?

        Returns:
            Data received from PL
            
        Raises:
            EngineTimeOut: On Timeout
            AuthenticationError: If checked for authentication
            EngineError
        
        """
        while self._toggle_2ps == self._buffer2ps.read() >> 2 & 0x1:
            continue
        
        helper = self._buffer2ps.read()
        if self._toggle_2ps == 1:
            self._toggle_2ps = 0
        else:
            self._toggle_2ps = 1
                
        if helper & 0x01:
            raise EngineError("Command aborted by hardware.")
        if (helper >> 1 & 0x01) == 0:
            raise EngineError("Got invalid data.")

        auth_status = helper >>3 & 0x1
        if auth and (auth_status == 0x0):
            raise AuthenticationError
        return (helper >> 16)
        
    def _send_data(self, data, type_val, eot=0, eoi=0, 
                   decrypt=0, a128=0, xof=0, hash_val=0):
        """
        Sends 32 bit of data and configuration to PL.
        
        Parameter:
            - data: 16 bit of data to send
            - type_val: Transmisson Type
            - eot: End of transmission Flag
            - eoi: End of input flag
            - decrypt: Decrypt?
            - a128: Use ASCON 128A?
            - xof: Use extendable Hash?
            - hash_val: Use hash?
            
        Returns:
            None
        
        Note:
            Normal operation is encryption (ASCON 128).
        """
        config = (type_val << 7) | (eot << 6) | (eoi << 5) | (decrypt << 4) | (a128 << 3) | (xof << 2) | (hash_val << 1) | (self._toggle_2pl)
        data_word = data << 16 |config
        self._buffer2pl.write(0x00, data_word)
        if self._toggle_2pl == 1:
            self._toggle_2pl = 0
        else:
            self._toggle_2pl = 1
    
    def _hash(self, msg: str, xof: bool, length: int=256) -> str:
        """ Internal implementation for ASCON Hash and Xof. 
        
        Parameters:
            - msg: Padded message as int
            - xof: True if xof
            - length: #Bits of hash message (defaults to 256)

        Returns:
            Hash of size 256 bits or Xof of arbirary length as hex string
        """
        self._set_toggle()
        
        if (xof):
            self._send_data(0x0, self.CONF, xof=1)
            self._wait_toggle()
            
            for i in range(2):
                data = (length >> (2 - (i+1)) * 16) & 0xFFFF
                if i==0:
                    self._send_data(data,type_val=self.LEN)
                else:
                     self._send_data(data,type_val=self.LEN, eot=1)
                self._wait_toggle()
        else:
            self._send_data(0x0, self.CONF, hash_val=1)
            self._wait_toggle()

        # Convert the hex string to an integer
        msg_int = int(msg, 16)

        # Loop to send 16 bits at a time
        for i in range(len(msg)//4):
            # Extract 16 bits from the message
            data = (msg_int >> (len(msg)//4 - (i+1)) * 16) & 0xFFFF

            # Determine if it's the last 16 bits
            eot = 1 if i == len(msg)//4-1 else 0

            # Send the data
            self._send_data(data, type_val=self.MSG, eot=eot)
            self._wait_toggle()


        # Receive Hash message
        result = 0
        for i in range(0, length, 16):
            data = 0x00        
            self._send_data(data, type_val=self.OK)
            helper = self._wait_toggle()
            result = result << 16 ^ helper
            
        self._send_data(0x00, self.OK)
        return hex(result)

    def _encrypt(self, plain: str, pad_len: int, ad_data: str, key: str, nonce: str, a128: bool=False) -> str:
        """
        Performs encryption in ASCON 128 and ASCON 128a.
        
        Parameters:
            - plain: Padded plaintext
            - len_pad: Length of padding in characters
            - ad_data: Padded additional data or None
            - tag: Tag from encryption
            - key: key for encryption
            - nonce: nonce for encryption
            - a128: Use ASCON-128A?
            
            
        Returns:
            Ciphertext as hex-string.
        """
        
        # Start communication with ASCON Engine and select mode
        self._set_toggle()
        if (a128):
            # Use ASCON 128a
            self._send_data(0x0, self.CONF, a128=1)
        else:
            # Use ASCON 128
            self._send_data(0x0, self.CONF)
        self._wait_toggle()

        # Send Key data
        key_int = int(key, 16)
        for i in range(len(key)//4):
            data = (key_int >> (len(key)//4 - (i+1)) * 16) & 0xFFFF
            eot = 1 if i == len(key)//4-1 else 0
            self._send_data(data, type_val=self.KEY, eot=eot)
            self._wait_toggle()
        
        # Send nonce data
        nonce_int = int(nonce, 16)
        for i in range(len(nonce)//4):
            data = (nonce_int >> (len(nonce)//4 - (i+1)) * 16) & 0xFFFF
            eot = 1 if i == len(nonce)//4-1 else 0
            self._send_data(data, type_val=self.NONCE, eot=eot)
            self._wait_toggle()
        
        # Send additional data if present
        
        if ad_data is not None:
            # Send additional data...
            ad_data_int = int(ad_data, 16)
            for i in range(len(ad_data)//4):
                data = (ad_data_int >> (len(ad_data)//4 - (i+1)) * 16) & 0xFFFF
                eot = 1 if i == len(ad_data)//4-1 else 0
                self._send_data(data, type_val=self.AD, eot=eot)
                self._wait_toggle()
        else:
            # Send skip additional data step
            self._send_data(0x00, type_val=self.SKIP_AD, eot=1)
            self._wait_toggle()
        
        # Send plaintext and receive ciphertext
        plain_int = int(plain, 16)
        cipher = 0
        
        blocks = (len(plain)//2)//8 # 1 block = 8 byte, 2 char = 1 byte
        block_len = 4

        if a128:
            blocks = (len(plain)//2)//16 # 1 block = 16 byte
            block_len = 8

        #print(f"Plaintext is: {plain}")
        for i in range(0, blocks):
            # Send plaintext MSB first
            for j in range(block_len):
                # Calculate the correct shift for MSB first
                shift = ((blocks - i - 1) * block_len + (block_len - j - 1)) * 16
                data = (plain_int >> shift) & 0xFFFF
                eot = 1 if i == blocks - 1 and j == block_len - 1 else 0
                self._send_data(data, type_val=self.PLAIN, eot=eot)
                self._wait_toggle()


            # Receive cipher
            for j in range(0, block_len):
                data = 0x00        
                self._send_data(data, type_val=self.OK)
                helper = self._wait_toggle()
                cipher = cipher << 16 ^ helper

        
        # Receive tag
        tag = 0
        for j in range(0, 8):
            data = 0x00        
            self._send_data(data, type_val=self.OK)
            helper = self._wait_toggle()
            tag = tag << 16 ^ helper
        
        self._send_data(data, type_val=self.OK)
        
        cipher = hex(cipher)[2:].zfill(len(plain))
        # Got cipher of correct length and leftovers
        out_cipher = cipher[:len(cipher)-pad_len]
        cipher_t = cipher[len(cipher)-pad_len:]
        
        return out_cipher, cipher_t, hex(tag)[2:].zfill(32)
    
    def _decrypt(self, cipher: str, pad_len: int, ad_data: str, tag:str, key: str, nonce: str, a128: bool=False) -> str | None:
        """
        Performs encryption in ASCON 128 and ASCON 128a.
        
        Parameter:
            - plain: Padded plaintext
            - ad_data: Padded additional data or None
            - tag: Tag from encryption
            - key: key for encryption
            - nonce: nonce for encryption
            - a128: Use ASCON-128A?
            
        Returns:
            Plaintext as hex-string or None if tag invalid.
        """
        
        # Start communication with ASCON Engine and select mode
        self._set_toggle()
        if (a128):
            # Use ASCON 128a
            self._send_data(0x0, self.CONF, decrypt=1, a128=1)
        else:
            # Use ASCON 128
            self._send_data(0x0, self.CONF, decrypt=1)
        self._wait_toggle()

        # Send 128 bit key data
        key_int = int(key, 16)
        for i in range(len(key)//4):
            data = (key_int >> (len(key)//4 - (i+1)) * 16) & 0xFFFF
            eot = 1 if i == len(key)//4-1 else 0
            self._send_data(data, type_val=self.KEY, eot=eot)
            self._wait_toggle()
        
        # Send 128 bit nonce data
        nonce_int = int(nonce, 16)
        for i in range(len(nonce)//4):
            data = (nonce_int >> (len(nonce)//4 - (i+1)) * 16) & 0xFFFF
            eot = 1 if i == len(nonce)//4-1 else 0
            self._send_data(data, type_val=self.NONCE, eot=eot)
            self._wait_toggle()
        
        # Send 128 bit tag data
        tag_int = int(tag, 16)
        for i in range(len(tag)//4):
            data = (tag_int >> (len(tag)//4 - (i+1)) * 16) & 0xFFFF
            eot = 1 if i == len(tag)//4-1 else 0
            self._send_data(data, type_val=self.TAG, eot=eot)
            self._wait_toggle()

        # Send additional data if present
        if ad_data is not None:
            ad_data_int = int(ad_data, 16)
            for i in range(len(ad_data)//4):
                data = (ad_data_int >> (len(ad_data)//4 - (i+1)) * 16) & 0xFFFF
                eot = 1 if i == len(ad_data)//4-1 else 0
                self._send_data(data, type_val=self.AD, eot=eot)
                self._wait_toggle()
        else:
            # Send skip additional data step
            self._send_data(0x00, type_val=self.SKIP_AD, eot=1)
            self._wait_toggle()
        
        # Send ciphertext and receive plaintext
        cipher_int = int(cipher, 16)
        plain = 0
        
        blocks = (len(cipher)//2)//8 # 1 block = 8 byte, 2 char = 1 byte
        block_len = 4

        if a128:
            blocks = (len(cipher)//2)//16 # 1 block = 16 byte
            block_len = 8

        for i in range(0, blocks):
            # Send ciphertext MSB first
            for j in range(block_len):
                shift = ((blocks - i - 1) * block_len + (block_len - j - 1)) * 16
                data = (cipher_int >> shift) & 0xFFFF
                eot = 1 if i == blocks - 1 and j == block_len - 1 else 0
                self._send_data(data, type_val=self.CIPHER, eot=eot)
                self._wait_toggle()


            # Receive cipher
            for j in range(0, block_len):
                data = 0x00        
                self._send_data(data, type_val=self.OK)
                helper = self._wait_toggle()
                plain = plain << 16 ^ helper

        
        # Receive auth
        data = 0x00        
        self._send_data(data, type_val=self.OK)
        try:
            helper = self._wait_toggle(auth=True)
        except AuthenticationError:
            self._send_data(data, type_val=self.OK)
            return "Tag invalid"

        self._send_data(data, type_val=self.OK)
        
        # Remove padding from plaintext
        plain =  hex(plain)[2:].zfill(len(cipher))
        # Got cipher of correct length and leftovers
        return plain[:len(plain)-pad_len]
        

    # PUBLIC METHODS
    def reset(self) -> None:
        """
        Performs soft reset for device
        """
        self._overlay = Overlay("./bitstreams/AsconEngine.bit")
        
        return
    
    def hard_reset(self) -> None:
        """
        Performs a hard reset by reprogramming FPGA.
        """
        self._overlay = Overlay("./bitstreams/AsconEngine.bit")
        
        in_buffer_addr = self._overlay.ip_dict['bd_in']['phys_addr']
        out_buffer_addr = self._overlay.ip_dict['bd_out']['phys_addr']
        
        self._buffer2pl = MMIO(in_buffer_addr, 8) # Buffer from PS to PL
        self._buffer2ps = MMIO(out_buffer_addr, 8) # Buffer from PL to PS


    def encrypt_128(self, plain: AsconPlain, key: AsconKey) -> AsconCipher:
        """
        Encrypts an plaintext using ASCON-128.
        
        Parameters:
            - plain: AsconPlain instance with plaintext and associated data
            - key: AsconKey instance with key and nonce to use (otherwise random)
        
        Returns:
            AsconCipher instance with ciphertext, associated data and tag.
        """    

        # Check input data
        if len(key.key) != 32:
            raise ValueError
        if len(key.nonce) != 32:
            raise ValueError
        
        # Add padding for input data
        if plain.is_hex:
            pad_plain, pad_len = Ascon.add_pad(plain.plain, self.BLOCK_SIZE_64)
        else:
            pad_plain, pad_len = Ascon.hex_and_pad(plain.plain, self.BLOCK_SIZE_64)
        
        if plain.ad_data is not None:
            if plain.is_hex:
                pad_ad_data, _ = Ascon.add_pad(plain.ad_data, self.BLOCK_SIZE_64)
            else:
                pad_ad_data, _ = Ascon.hex_and_pad(plain.ad_data, self.BLOCK_SIZE_64)            
        else:
            pad_ad_data = None
        
        cipher, cipher_t, tag = self._encrypt(pad_plain, pad_len, pad_ad_data,
                                    key.key, key.nonce)
        
        # Create cipher element to return
        result = AsconCipher(cipher=cipher,
                             tag=tag,
                             ad_data=plain.ad_data,
                             cipher_t=cipher_t,
                             is_hex=plain.is_hex)
        return result

    def encrypt_128a(self, plain: AsconPlain, key: AsconKey=None) -> AsconCipher:
        """
        Encrypts an plaintext using ASCON-128a.
        
        Parameters:
            - plain: AsconPlain instance with plaintext and associated data
            - key: AsconKey instance with key and nonce to use (otherwise random)
        
        Returns:
            AsconCipher instance with ciphertext, associated data and tag.
        """
        # Check input data
        if len(key.key) != 32:
            raise ValueError
        if len(key.nonce) != 32:
            raise ValueError

        # Add padding for input data
        if plain.is_hex:
            pad_plain, pad_len = Ascon.add_pad(plain.plain, self.BLOCK_SIZE_128)
        else:
            pad_plain, pad_len = Ascon.hex_and_pad(plain.plain, self.BLOCK_SIZE_128)
        
        if plain.ad_data is not None:
            if plain.is_hex:
                pad_ad_data, _ = Ascon.add_pad(plain.ad_data, self.BLOCK_SIZE_128)
            else:
                pad_ad_data, _ = Ascon.hex_and_pad(plain.ad_data, self.BLOCK_SIZE_128)
        else:
            pad_ad_data = None

        cipher, cipher_t, tag = self._encrypt(pad_plain, pad_len, pad_ad_data,
                                    key.key, key.nonce, a128=True)

        # Create cipher element to return
        result = AsconCipher(cipher=cipher,
                             tag=tag,
                             ad_data=plain.ad_data,
                             cipher_t=cipher_t,
                             is_hex=plain.is_hex)
        return result

    def decrypt_128(self, cipher: AsconCipher, key: AsconKey) -> str|None:
        """
        Decrypts a ciphertext with provided key using ASCON-128.
        
        Parameters:
            - cipher: AsconCipher instance with ciphertext, associated data andn text
            - key: AsconKey instance with key and nonce to use
        
        Returns:
            String with plaintext or None on error.
        """
        # Check input data
        if len(key.key) != 32:
            raise ValueError
        if len(key.nonce) != 32:
            raise ValueError
        if len(cipher.tag) != 32:
            raise ValueError

        # Add padding for input data & create complete cipher text
        complete_ciper = cipher.cipher + cipher.cipher_t
        
        ad_data = cipher.ad_data
        if ad_data is not None:
            if cipher.is_hex:
                ad_data, _ = Ascon.add_pad(ad_data, self.BLOCK_SIZE_64)
            else:
                ad_data, _ = Ascon.hex_and_pad(ad_data, self.BLOCK_SIZE_64)

        plain = self._decrypt(complete_ciper, len(cipher.cipher_t), ad_data, cipher.tag,
                              key.key, key.nonce)
        
        return plain

    def decrypt_128a(self, cipher: AsconCipher, key: AsconKey) -> str|None:
        """
        Decrypts a ciphertext with provided key using ASCON-128a.
        
        Parameters:
            - cipher: AsconCipher instance with ciphertext, associated data and
                      text
            - key: AsconKey instance with key and nonce to use
        
        Returns:
            String with plaintext or None on error.
            
        """
        # Check input data
        if len(key.key) != 32:
            raise ValueError
        if len(key.nonce) != 32:
            raise ValueError
        if len(cipher.tag) != 32:
            raise ValueError

        # Add padding for input data & create complete cipher text
        complete_ciper = cipher.cipher + cipher.cipher_t
        
        ad_data = cipher.ad_data
        if ad_data is not None:
            if cipher.is_hex:
                ad_data, _ = Ascon.add_pad(ad_data, self.BLOCK_SIZE_128)
            else:
                ad_data, _ = Ascon.hex_and_pad(ad_data, self.BLOCK_SIZE_128)

        plain = self._decrypt(complete_ciper, len(cipher.cipher_t), ad_data, cipher.tag,
                              key.key, key.nonce, a128=True)
        
        return plain
        
    def get_hash(self, msg: str, is_hex: bool=False) -> str:
        """
        Returns ASCON-HASH for msg.
        
        Parameters:
            - msg: Message to hash (hex-string)

        Returns:
            256 bit ASCON-Hash in hex-string representation.
        
        """
        if is_hex:
            msg, _ = Ascon.add_pad(msg, self.BLOCK_SIZE_64)
        else:
            msg, _ = Ascon.hex_and_pad(msg, self.BLOCK_SIZE_64)

        str_hash = self._hash(msg, xof=False)
        return str_hash[2:].zfill(64) # Add padding to 64 characters (32 bytes)

    def get_xof(self, msg: str, length: int, is_hex: bool=False) -> str:
        """
        Returns ASCON-XoF for msg.
        
        Parameters:
            - msg: Message to hash (hex-string)
            - length: Number of bytes to retrive (max: (2^32) - 1)

        Returns:
            Arbitrary length ASCON-XoF in hex-string representation.
        
        """
        len_chars = length*8
        if is_hex:
            msg, _= Ascon.add_pad(msg, self.BLOCK_SIZE_64)
        else:
            msg, _= Ascon.hex_and_pad(msg, self.BLOCK_SIZE_64)
        xof =  self._hash(msg, xof=True, length=len_chars)
        return xof[2:].zfill(length*2)


This is a usage example for for ASCON Engine.

We will perform the following computations:

1. Ascon-Hash
 1. Hash with empty string
 2.  Hash with non empty string
2. Ascon-XoF
 1. Ascon-Xof with empty string
 2. Ascon-Xof with non empty string
3. ASCON-128
 1. ASCON-128 Encryption without additional data
 2. ASCON-128 Encryption with additional data
 3. ASCON-128 Encryption with additonal data, longer input
 4. ASCON-128 Decryption without additional data
 5. ASCON-128 Decryption with additional data
 6. ASCON-128 Decryption with additonal data, longer input
 7. ASCON-128 Decryption without additional data, incorrect tag
 8. ASCON-128 Decryption with additional data, incorrect tag
4. ASCON-128A
 1. ASCON-128A Encryption without additional data
 2. ASCON-128A Encryption with additional data
 3. ASCON-128A Enctyption with additonal data, longer input
 4. ASCON-128A Decryption without additional data
 5. ASCON-128A Decryption with additional data
 6. ASCON-128A Decryption with additonal data, longer input
 7. ASCON-128A Decryption without additional data, incorrect tag
 8. ASCON-128A Decryption with additional data, incorrect tag

To verify correctness of results feel free to use https://hashing.tools/ascon for Ascon-Hash and Ascon-Xof use any of the implementations recommended by https://ascon.iaik.tugraz.at/implementations.html for Ascon-128 and Ascon-128a. A python based implementation can be found at https://github.com/meichlseder/pyascon.
 

In [220]:
# Create key representation
my_key = AsconKey(key="000102030405060708090A0B0C0D0E0F",
                  nonce="000102030405060708090A0B0C0D0E0F")

# Create plaintext representations

plain_128 = AsconPlain(plain="", ad_data=None)

plain_128_ad = AsconPlain(plain="", ad_data="\0")

plain_long_128 = AsconPlain(plain="000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F",
                            ad_data="000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F",
                            is_hex=True)

# The following statements can be used as golden data
# Note: This elements do not contain a cipher_t element and might
#       infer incorrect decryption results.

#long_128a_ad = AsconCipher(cipher="A382BC87C68946D86A921DD88E2ADDDFBBE77D4112830E01960B9D38D5",
#                           tag="A55236AC020DBDA74CE6CCD10C68C4D8514450",
#                           ad_data="000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F")
#cipher_128 = AsconCipher(cipher="3c830fbef3a1651b",
#                         tag="e355159f292911f794cb1432a0103a8a",
#                         ad_data = None)
#cipher_128_ad = AsconCipher(cipher="3d4742c7de2afc51",
#                            tag="944DF887CD4901614C5DEDBC42FC0DA0",
#                            ad_data = "\0")
#cipher_128a = AsconCipher(cipher="ee480efdd1b652606f3c06d33047c1b2",
#                          tag="7A834E6F09210957067B10FD831F0078",
#                          ad_data = None)
#cipher_128a_ad = AsconCipher(cipher="692c2866caec7478baf5c0917eb27611",
#                             tag="AF3031B07B129EC84153373DDCABA528",
#                             ad_data = "\0")

# Creatae Ascon-Engine
ascon_engine = Ascon()

# Examples for Hash method
hash_0 = ascon_engine.get_hash(msg="")
hash_1 = ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog")

print(f"Results for Ascon-Hash:\n1.\t{hash_0}\n2.\t{hash_1}")

# Examples for Xof method
xof_0 = ascon_engine.get_xof(msg="", length=32)
xof_1 = ascon_engine.get_xof(msg="The quick brown fox jumps over the lazy dog", length=32)

print(f"\n\nResults for Ascon-Xof:\n1.\t{xof_0}\n2.\t{xof_1}")

# Examples for Ascon-128
# Note: As we encrypt a null string, there will be no ciphertext, only a tag.
#       Same applies for the decryption. It will only yield correct tag result.
cipher_128 = ascon_engine.encrypt_128(plain_128, my_key)
cipher_128_ad = ascon_engine.encrypt_128(plain_128_ad, my_key)
cipher_128_long = ascon_engine.encrypt_128(plain_long_128, my_key)

cipher_128_faulty = AsconCipher(cipher=cipher_128.cipher,
                                tag=cipher_128.tag[:-16] + "0000000000000000",
                                ad_data=cipher_128.ad_data,
                                cipher_t=cipher_128.cipher_t,
                                is_hex=cipher_128.is_hex)
                                 
cipher_128_ad_faulty = AsconCipher(cipher=cipher_128_ad.cipher,
                                   tag=cipher_128_ad.tag[:-16] + "0000000000000000",
                                   ad_data=cipher_128_ad.ad_data,
                                   cipher_t=cipher_128_ad.cipher_t,
                                   is_hex=cipher_128_ad.is_hex)

plain_0 = ascon_engine.decrypt_128(cipher_128, my_key)
plain_1 = ascon_engine.decrypt_128(cipher_128_ad, my_key)
plain_2 = ascon_engine.decrypt_128(cipher_128_long, my_key)
plain_3 = ascon_engine.decrypt_128(cipher_128_faulty, my_key)
plain_4 = ascon_engine.decrypt_128(cipher_128_ad_faulty, my_key)

print("\n\nResults for Ascon-128:\n")
print(f"1.\tCipher= {cipher_128.cipher}\n\tTag= {cipher_128.tag}")
print(f"2.\tCipher= {cipher_128_ad.cipher}\n\tTag= {cipher_128_ad.tag}")
print(f"3.\tCipher= {cipher_128_long.cipher}\n\tTag= {cipher_128_long.tag}")
print(f"4.\tPlaintext= {plain_0}")
print(f"5.\tPlaintext= {plain_1}")
print(f"6.\tPlaintext= {plain_2}")
print(f"7.\tPlaintext= {plain_3}")
print(f"8.\tPlaintext= {plain_4}")

# Examples for Ascon-128A
cipher_128a = ascon_engine.encrypt_128a(plain_128, my_key)
cipher_128a_ad = ascon_engine.encrypt_128a(plain_128_ad, my_key)
cipher_128a_long = ascon_engine.encrypt_128a(plain_long_128, my_key)


cipher_128a_faulty = AsconCipher(cipher=cipher_128a.cipher,
                                 tag=cipher_128a.tag[:-16] + "0000000000000000",
                                 ad_data=cipher_128a.ad_data,
                                 cipher_t=cipher_128a.cipher_t,
                                 is_hex=cipher_128a.is_hex)
                                 
cipher_128a_ad_faulty = AsconCipher(cipher=cipher_128a_ad.cipher,
                                    tag=cipher_128a_ad.tag[:-16] + "0000000000000000",
                                    ad_data=cipher_128a_ad.ad_data,
                                    cipher_t=cipher_128a_ad.cipher_t,
                                    is_hex=cipher_128a_ad.is_hex)

plain_5 = ascon_engine.decrypt_128a(cipher_128a, my_key)
plain_6 = ascon_engine.decrypt_128a(cipher_128a_ad, my_key)
plain_7 = ascon_engine.decrypt_128a(cipher_128a_long, my_key)
plain_8 = ascon_engine.decrypt_128a(cipher_128a_faulty, my_key)
plain_9 = ascon_engine.decrypt_128a(cipher_128a_ad_faulty, my_key)

print("\n\nResults for Ascon-128A:")
print(f"1.\tCipher= {cipher_128a.cipher}\n\tTag= {cipher_128a.tag}")
print(f"2.\tCipher= {cipher_128a_ad.cipher}\n\tTag= {cipher_128a_ad.tag}")
print(f"3.\tCipher= {cipher_128a_long.cipher}\n\tTag= {cipher_128a_long.tag}")
print(f"4.\tPlaintext= {plain_5}")
print(f"5.\tPlaintext= {plain_6}")
print(f"6.\tPlaintext= {plain_7}")
print(f"7.\tPlaintext= {plain_8}")
print(f"8.\tPlaintext= {plain_9}")


Results for Ascon-Hash:
1.	7346bc14f036e87ae03d0997913088f5f68411434b3cf8b54fa796a80d251f91
2.	3375fb43372c49cbd48ac5bb6774e7cf5702f537b2cf854628edae1bd280059e


Results for Ascon-Xof:
1.	5d4cbde6350ea4c174bd65b5b332f8408f99740b81aa02735eaefbcf0ba0339e
2.	c100696bd70a3e731873bdc8a76ffb53b6cca80b694473b320d436883bbbc300


Results for Ascon-128:

1.	Cipher= 
	Tag= e355159f292911f794cb1432a0103a8a
2.	Cipher= 
	Tag= 944df887cd4901614c5dedbc42fc0da0
3.	Cipher= b96c78651b6246b0c3b1a5d373b0d5168dca4a96734cf0ddf5f92f8d15e30270
	Tag= 279bf6a6cc3f2fc9350b915c292bdb8d
4.	Plaintext= 
5.	Plaintext= 
6.	Plaintext= 000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f
7.	Plaintext= Tag invalid
8.	Plaintext= Tag invalid


Results for Ascon-128:
1.	Cipher= 
	Tag= 7a834e6f09210957067b10fd831f0078
2.	Cipher= 
	Tag= af3031b07b129ec84153373ddcaba528
3.	Cipher= a55236ac020dbda74ce6ccd10c68c4d8514450a382bc87c68946d86a921dd88e
	Tag= 2adddfbbe77d4112830e01960b9d38d5
4.	Plaintext= 
5.	Plaintext= 
6.	

In [221]:
# Here are some more examples

print(ascon_engine.get_xof(msg="The quick borwn fox jumps over the lazy dog",length=128) +"\n")
print(ascon_engine.get_xof(msg="The quick brown fox jumps over the lazy dog.",length=128)+"\n")
print(ascon_engine.get_xof(msg="The quick brown fox jumps over the lazy dog..",length=128)+"\n")
print(ascon_engine.get_xof(msg="The quick brown fox jumps over the lazy dog...",length=128)+"\n")
print(ascon_engine.get_xof(msg="The quick brown fox jumps over the lazy dog....",length=128)+"\n")

print(ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog")+"\n")
print(ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog.")+"\n")
print(ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog..")+"\n")
print(ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog...")+"\n")
print(ascon_engine.get_hash(msg="The quick brown fox jumps over the lazy dog....")+"\n")


plain_utf_8 = AsconPlain(plain="The quick brown fox jumps over the lazy dog",
                         ad_data="The quick brown fox jumps over the lazy dog")

cipher_128 = ascon_engine.encrypt_128(plain_utf_8, my_key)
cipher_128a = ascon_engine.encrypt_128a(plain_utf_8, my_key)

print(f"Ascon-128:\n\tCipher= {cipher_128.cipher}\n\tTag= {cipher_128.tag}")
print(f"Ascon-128a.\tCipher= {cipher_128a.cipher}\n\tTag= {cipher_128a.tag}")


4bddae049d645e845bd92da97634db23ad6fd29b1079ae6d58126d353961081a7db3e9e709dcd864052b88a96c84060e7bb9ad6e4e37b8056638d55ffc394c095c64394c2b13a78f68570035e86edb35c7d2d8d53670a3b5633617ab2263084e989699de87860436e5a75a81d6c940fba85ac585f77c021527efc74d4d2d1604

6924509854c1a1aa9f66d4eb2c1fb05c8f93456fab2fa7f037a4f541bf1a04ee8cce2d526e8cf374992ea02b2414507d46a6c1f45b3a00713b7516e48b3353e73c84b43ff4437bec29e0e15b88f5155aebc9ffb38c8d1a40e8a352fed90bb549944ff28f9198fde8c44d1275573edb942b5588616ea2402957b4d28b7b4120b9

24ab449c2c053af2567541a3d3adbb6d3173a507b20d350d3a35ea649b55178e3e67edf300fa92d84cc87d2a997131d6ae886407e1e94d5821474e546192b50d603fafbdf5844e1f57b8a6ac55e2defdfa0408a66a0e0a8100565dc1be7cbabe551b35faaa6a7b8a3638872140c6d797a1ca20d5e838f7bcc4f3da1fa551106f

25f4d3154ecaa7ebd1e725d2b5efb5352d4912267da587f82e21085cd392f23d1c324553e732ad19c6e6f343114e9a293fdecac0acccef91df87741fb0f470689e924986f4d91819871844693c24efaab78963b10f95fb4bddfbb0ddf1315ef435b70f3c684a2f86246e11104f5698a211

# Testvectors for Ascon

There is no need to run the code if you do not want to verify the corectness of my implementation.

The following code blocks provide testvectors for Ascon-Hash, Ascon-Xof, Ascon-128 
and Ascon-128a.

Note: This code is taken from https://github.com/meichlseder/pyascon and licenced under CC0.

In [222]:
# This part of the code is licenced under CC0 1.0 Universal.
# Licence information can be obtainedn on
# https://github.com/meichlseder/pyascon/blob/master/LICENSE
# or in the licence block below.

"""
Writers for output test vectors in Text and JSON formats.
"""


class TextWriter:
    """
    TextWriter produces an array of key-value objects.
    """

    def __init__(self, filename):
        self.fp = open(filename + ".txt", "w")
        self.is_open = False

    def __enter__(self):
        return self

    def __exit__(self, stype, value, traceback):
        pass

    def append(self, label, value, length=None):
        assert self.is_open, "cannot append if not open yet"
        if length is not None:
            assert len(value) >= length
            value = value[:length].hex().upper()
        self.fp.write("{} = {}\n".format(label, value))

    def open(self):
        assert not self.is_open, "cannot open twice"
        self.is_open = True

    def close(self):
        assert self.is_open, "cannot close if not open first"
        self.fp.write("\n")
        self.is_open = False


class JSONWriter:
    """
    JSONWriter produces an array of JSON objects.
    """

    def __init__(self, filename):
        self.level = 1
        self.fp = open(filename + ".json", "w")
        self.has_item = False
        self.tab = " " * 2
        self.comma = lambda: "," * self.has_item
        self.ws = lambda: "\n" * \
            (self.level > 0 or self.has_item) + self.tab * self.level
        self.fp.write("[")

    def __enter__(self):
        return self

    def __exit__(self, stype, value, traceback):
        self.level -= 1
        self.fp.write("{}]\n".format(self.ws()))

    def append(self, label, value, length=None):
        if length is not None:
            assert len(value) >= length
            value = '"{}"'.format(value[:length].hex().upper())
        self.fp.write('{}{}"{}": {}'.format(
            self.comma(), self.ws(), label, value))
        self.has_item = True

    def open(self):
        assert (self.level > 0 or not self.has_item)
        self.fp.write("{}{}{{".format(self.comma(), self.ws()))
        self.level += 1
        self.has_item = False

    def close(self):
        assert (self.level > 0 or not self.has_item)
        self.level -= 1
        self.fp.write("{}}}".format(self.ws()))
        self.has_item = True


class MultipleWriter:
    """
    Merge multiple writers to ease invocation.
    """

    def __init__(self, filename):
        self.writers = [JSONWriter(filename), TextWriter(filename)]

    def __enter__(self):
        for w in self.writers:
            w.__enter__()
        return self

    def __exit__(self, stype, value, traceback):
        for w in self.writers:
            w.__exit__(stype, value, traceback)
            w.fp.close()

    def open(self):
        for w in self.writers:
            w.open()

    def append(self, label, value, length=None):
        for w in self.writers:
            w.append(label, value, length)

    def close(self):
        for w in self.writers:
            w.close()

In [223]:
# This part of the code is licenced under CC0 1.0 Universal.
# Licence information can be obtainedn on
# https://github.com/meichlseder/pyascon/blob/master/LICENSE
# or in the licence block below.

"""
KAT implementation for NIST (based on TestVectorGen.zip)
"""

import sys


def kat_bytes(length):
    return bytes(bytearray([i % 256 for i in range(length)]))


def kat_aead(variant):
    MAX_MESSAGE_LENGTH = 32
    MAX_ASSOCIATED_DATA_LENGTH = 32

    klen = 16  # =CRYPTO_KEYBYTES
    nlen = 16  # =CRYPTO_NPUBBYTES
    tlen = 16  # <=CRYPTO_ABYTES
    filename = "LWC_AEAD_KAT_{klenbits}_{nlenbits}".format(klenbits=klen*8, nlenbits=nlen*8)
    assert variant in ["Ascon-128", "Ascon-128a"]

    key   = kat_bytes(klen)
    nonce = kat_bytes(nlen)
    msg   = kat_bytes(MAX_MESSAGE_LENGTH)
    ad    = kat_bytes(MAX_ASSOCIATED_DATA_LENGTH)
    
    ascon_engine = Ascon()
    
    with MultipleWriter(filename) as w:
        count = 1
        for mlen in range(MAX_MESSAGE_LENGTH+1):
            for adlen in range(MAX_ASSOCIATED_DATA_LENGTH+1):
                #print(f"Count is: {count}")
                w.open()
                w.append("Count", count)
                count += 1
                w.append("Key", key, klen)
                w.append("Nonce", nonce, nlen)
                w.append("PT", msg, mlen)
                w.append("AD", ad, adlen)
                
                # Set key and plaintext element for Ascon Engine
                my_key = AsconKey(key=key.hex(),
                                  nonce=nonce.hex())

                if(adlen > 0):
                    my_plain = AsconPlain(plain=msg[:mlen].hex(),
                                          ad_data=ad[:adlen].hex(),
                                          is_hex=True)
                else:
                    my_plain = AsconPlain(plain=msg[:mlen].hex(),
                                          ad_data=None,
                                          is_hex=True)
                cipher = None
                if variant == "Ascon-128":
                    cipher = ascon_engine.encrypt_128(my_plain, my_key)
                    plain = ascon_engine.decrypt_128(cipher, my_key)

                elif variant == "Ascon-128a":
                    cipher = ascon_engine.encrypt_128a(my_plain, my_key)
                    plain = ascon_engine.decrypt_128a(cipher, my_key)
                
                
                ct = bytes.fromhex(cipher.cipher) + bytes.fromhex(cipher.tag)
                
                assert len(ct) == mlen + tlen
                w.append("CT", ct, len(ct))
                    
                msg2 = bytes.fromhex(plain)
                #print(msg2)
                assert len(msg2) == mlen
                assert msg2 == msg[:mlen]
                w.close()


def kat_hash(variant="Ascon-Hash"):
    MAX_MESSAGE_LENGTH = 1024
    hlen = 32  # =CRYPTO_BYTES
    filename = "LWC_HASH_KAT_{hlenbits}".format(hlenbits=hlen*8)
    assert variant in ["Ascon-Xof", "Ascon-Hash"]

    ascon_engine = Ascon()

    msg = kat_bytes(MAX_MESSAGE_LENGTH)
    with MultipleWriter(filename) as w:
        count = 1
        for mlen in range(MAX_MESSAGE_LENGTH+1):
            w.open()
            w.append("Count", count)
            count += 1
            w.append("Msg", msg, mlen)
                        
            if variant == "Ascon-Hash":
                tag = ascon_engine.get_hash(msg[:mlen].hex(), is_hex=True)
            elif variant == "Ascon-Xof":
                tag = ascon_engine.get_xof(msg[:mlen].hex(), hlen, is_hex=True)
                
            tag = bytes.fromhex(tag)
            w.append("MD", tag, hlen)
            w.close()



def kat(variant):
    aead_variants = ["Ascon-128", "Ascon-128a"]
    hash_variants = ["Ascon-Hash", "Ascon-Xof"]
    assert variant in aead_variants + hash_variants
    if variant in aead_variants: kat_fun = kat_aead
    if variant in hash_variants: kat_fun = kat_hash
    kat_fun(variant)


Creative Commons Legal Code<br>
<br>
CC0 1.0 Universal

    CREATIVE COMMONS CORPORATION IS NOT A LAW FIRM AND DOES NOT PROVIDE
    LEGAL SERVICES. DISTRIBUTION OF THIS DOCUMENT DOES NOT CREATE AN
    ATTORNEY-CLIENT RELATIONSHIP. CREATIVE COMMONS PROVIDES THIS
    INFORMATION ON AN "AS-IS" BASIS. CREATIVE COMMONS MAKES NO WARRANTIES
    REGARDING THE USE OF THIS DOCUMENT OR THE INFORMATION OR WORKS
    PROVIDED HEREUNDER, AND DISCLAIMS LIABILITY FOR DAMAGES RESULTING FROM
    THE USE OF THIS DOCUMENT OR THE INFORMATION OR WORKS PROVIDED
    HEREUNDER.

Statement of Purpose<br>
<br>
The laws of most jurisdictions throughout the world automatically confer
exclusive Copyright and Related Rights (defined below) upon the creator
and subsequent owner(s) (each and all, an "owner") of an original work of
authorship and/or a database (each, a "Work").<br>

Certain owners wish to permanently relinquish those rights to a Work for
the purpose of contributing to a commons of creative, cultural and
scientific works ("Commons") that the public can reliably and without fear
of later claims of infringement build upon, modify, incorporate in other
works, reuse and redistribute as freely as possible in any form whatsoever
and for any purposes, including without limitation commercial purposes.
These owners may contribute to the Commons to promote the ideal of a free
culture and the further production of creative, cultural and scientific
works, or to gain reputation or greater distribution for their Work in
part through the use and efforts of others.<br>

For these and/or other purposes and motivations, and without any
expectation of additional consideration or compensation, the person
associating CC0 with a Work (the "Affirmer"), to the extent that he or she
is an owner of Copyright and Related Rights in the Work, voluntarily
elects to apply CC0 to the Work and publicly distribute the Work under its
terms, with knowledge of his or her Copyright and Related Rights in the
Work and the meaning and intended legal effect of CC0 on those rights.<br>

1. Copyright and Related Rights. A Work made available under CC0 may be
protected by copyright and related or neighboring rights ("Copyright and
Related Rights"). Copyright and Related Rights include, but are not
limited to, the following:<br>

  i. the right to reproduce, adapt, distribute, perform, display,
     communicate, and translate a Work;<br>
 ii. moral rights retained by the original author(s) and/or performer(s);<br>
iii. publicity and privacy rights pertaining to a person's image or
     likeness depicted in a Work;<br>
 iv. rights protecting against unfair competition in regards to a Work,
     subject to the limitations in paragraph 4(a), below;
  v. rights protecting the extraction, dissemination, use and reuse of data
     in a Work;<br>
 vi. database rights (such as those arising under Directive 96/9/EC of the
     European Parliament and of the Council of 11 March 1996 on the legal
     protection of databases, and under any national implementation
     thereof, including any amended or successor version of such
     directive); and<br>
vii. other similar, equivalent or corresponding rights throughout the
     world based on applicable law or treaty, and any national
     implementations thereof.<br>
<br>
2. Waiver. To the greatest extent permitted by, but not in contravention
of, applicable law, Affirmer hereby overtly, fully, permanently,
irrevocably and unconditionally waives, abandons, and surrenders all of
Affirmer's Copyright and Related Rights and associated claims and causes
of action, whether now known or unknown (including existing as well as
future claims and causes of action), in the Work (i) in all territories
worldwide, (ii) for the maximum duration provided by applicable law or
treaty (including future time extensions), (iii) in any current or future
medium and for any number of copies, and (iv) for any purpose whatsoever,
including without limitation commercial, advertising or promotional
purposes (the "Waiver"). Affirmer makes the Waiver for the benefit of each
member of the public at large and to the detriment of Affirmer's heirs and
successors, fully intending that such Waiver shall not be subject to
revocation, rescission, cancellation, termination, or any other legal or
equitable action to disrupt the quiet enjoyment of the Work by the public
as contemplated by Affirmer's express Statement of Purpose.<br>
<br>
3. Public License Fallback. Should any part of the Waiver for any reason
be judged legally invalid or ineffective under applicable law, then the
Waiver shall be preserved to the maximum extent permitted taking into
account Affirmer's express Statement of Purpose. In addition, to the
extent the Waiver is so judged Affirmer hereby grants to each affected
person a royalty-free, non transferable, non sublicensable, non exclusive,
irrevocable and unconditional license to exercise Affirmer's Copyright and
Related Rights in the Work (i) in all territories worldwide, (ii) for the
maximum duration provided by applicable law or treaty (including future
time extensions), (iii) in any current or future medium and for any number
of copies, and (iv) for any purpose whatsoever, including without
limitation commercial, advertising or promotional purposes (the
"License"). The License shall be deemed effective as of the date CC0 was
applied by Affirmer to the Work. Should any part of the License for any
reason be judged legally invalid or ineffective under applicable law, such
partial invalidity or ineffectiveness shall not invalidate the remainder
of the License, and in such case Affirmer hereby affirms that he or she
will not (i) exercise any of his or her remaining Copyright and Related
Rights in the Work or (ii) assert any associated claims and causes of
action with respect to the Work, in either case contrary to Affirmer's
express Statement of Purpose.<br>
<br>
4. Limitations and Disclaimers.<br>

 a. No trademark or patent rights held by Affirmer are waived, abandoned,
    surrendered, licensed or otherwise affected by this document.<br>
 b. Affirmer offers the Work as-is and makes no representations or
    warranties of any kind concerning the Work, express, implied,
    statutory or otherwise, including without limitation warranties of
    title, merchantability, fitness for a particular purpose, non
    infringement, or the absence of latent or other defects, accuracy, or
    the present or absence of errors, whether or not discoverable, all to
    the greatest extent permissible under applicable law.<br>
 c. Affirmer disclaims responsibility for clearing rights of other persons
    that may apply to the Work or any use thereof, including without
    limitation any person's Copyright and Related Rights in the Work.
    Further, Affirmer disclaims responsibility for obtaining any necessary
    consents, permissions or other rights required for any use of the
    Work.<br>
 d. Affirmer understands and acknowledges that Creative Commons is not a
    party to this document and has no duty or obligation with respect to
    this CC0 or use of the Work.<br>

In [224]:
# Start test vectors
kat("Ascon-128")
#kat("Ascon-128a")
#kat("Ascon-Hash")
#kat("Ascon-Xof")